In [4]:
import importlib

In [5]:
%%time
import auxLib; importlib.reload(auxLib)

Importing library...
Library successfully imported
Wall time: 226 ms


<module 'auxLib' from 'd:\\Facultad\\Doctorado\\Proyectos\\paper\\auxLib.py'>

# Dense Neural Networks

In [6]:
L = 40

In [7]:
# All possible configurations

data = auxLib.SyntheticImagesGen(training=['ferro','neel','stripe'], L=L)
train_images, train_labels = data.dataGenerator(10000)
test_images, test_labels = data.dataGenerator(1000)

Generating synthetic data...
Done!
Elapsed time: 0.7687182426452637 seconds
Generating synthetic data...
Done!
Elapsed time: 0.03137350082397461 seconds


In [8]:
# Dense neural network

dnn = auxLib.DenseNeuralNetworkGen(input_shape=L*L, 4, layers[
    {'type': 'dense', 'units': 512, 'activation': 'relu'},
    {'type': 'dense', 'units': 4, 'activation': 'softmax'}
])

dnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dnn.summary()
dnn.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

SyntaxError: positional argument follows keyword argument (Temp/ipykernel_12692/3948336229.py, line 6)